In [1]:
from kafka import KafkaProducer
# from confluent_kafka import Producer
from time import sleep
from json import dumps
import sys
# sys.path.append('D:/Projects/Crypto/Scripts/API')
sys.path.extend([
    'D:/Projects/Crypto/Scripts/API',
    'D:/Projects/Crypto/Scripts/Calculations'
])
import platforms as pl
import Exchanges_list as el
import Current_Date_Time as dt

In [2]:
data = pl.coins_list()
print(type(data))

<class 'list'>


In [8]:
#Creating custom Exception
class APICallError(Exception):
    """Raised when there is an error during an API call"""
    pass
flag = 0
coins_list = []
while True:
    if flag ==0:
        try:
            
            # Fetch data from API
            data = pl.coins_list()
            for i in range (0, len(data)):
                coins_list.append(data[i]['code'])

            if 'error' in data:
                raise APICallError(f"API Error: {data['error']['description']}")

        except APICallError as api_err:
            print(f"API call failed: {api_err}")

        else:
            producer = KafkaProducer(bootstrap_servers=['18.217.211.255:9092'],
                                    value_serializer=lambda x: dumps(x).encode('utf-8'))
            producer.send('crypto', value=data)
            # print("Coins")
        flag =1
        sleep(3)
    
    else:
        for coin in coins_list:
            try:
            # Fetch data from API
                data = el.exchanges(coin)
                if 'error' in data:
                    raise APICallError(f"API Error: {data['error']['description']}")

            except APICallError as api_err:
                print(f"API call failed: {api_err}")

            else:
                producer = KafkaProducer(bootstrap_servers=['18.217.211.255:9092'],
                                value_serializer=lambda x: dumps(x).encode('utf-8'))
                producer.send('crypto', value=data)
                flag = 0
                sleep(2)
                coins_list = []

['BTC', 'ETH', 'USDT']
['BTC', 'ETH', 'USDT']


KeyboardInterrupt: 